In [1]:
pip install tensorflow opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [3]:
import pandas as pd
train_data = pd.read_csv("train.csv")
train_data.head()



,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_data = pd.read_csv("test.csv")
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split
train_labels = train_data['label']
train_features = train_data.drop('label', axis=1)


In [6]:
# Normalize pixel values to the range [0, 1]
train_features = train_features / 255.0

In [7]:
# Split train data into train and validation sets
train_features, val_features, train_labels, val_labels = train_test_split(
    train_features, train_labels, test_size=0.1, random_state=42
)

In [8]:
 #Reshape features to match image dimensions
train_features = train_features.values.reshape(-1, 28, 28, 1)
val_features = val_features.values.reshape(-1, 28, 28, 1)

In [9]:
# Build the neural network model
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])


In [10]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train the model
model.fit(train_features, train_labels, validation_data=(val_features, val_labels), epochs=10, batch_size=64)


Epoch 1/10
591/591 [==============================] - 15s 24ms/step - loss: 0.3698 - accuracy: 0.8873 - val_loss: 0.1217 - val_accuracy: 0.9638
Epoch 2/10
591/591 [==============================] - 13s 23ms/step - loss: 0.1386 - accuracy: 0.9600 - val_loss: 0.0791 - val_accuracy: 0.9760
Epoch 3/10
591/591 [==============================] - 15s 25ms/step - loss: 0.0971 - accuracy: 0.9713 - val_loss: 0.0580 - val_accuracy: 0.9826
Epoch 4/10
591/591 [==============================] - 14s 24ms/step - loss: 0.0817 - accuracy: 0.9746 - val_loss: 0.0540 - val_accuracy: 0.9831
Epoch 5/10
591/591 [==============================] - 15s 25ms/step - loss: 0.0689 - accuracy: 0.9790 - val_loss: 0.0515 - val_accuracy: 0.9843
Epoch 6/10
591/591 [==============================] - 14s 24ms/step - loss: 0.0607 - accuracy: 0.9812 - val_loss: 0.0446 - val_accuracy: 0.9850
Epoch 7/10
591/591 [==============================] - 14s 24ms/step - loss: 0.0510 - accuracy: 0.9838 - val_loss: 0.0493 - val_accuracy:

In [12]:
# Preprocess test dataset
test_features = test_data / 255.0
test_features = test_features.values.reshape(-1, 28, 28, 1)

In [13]:
 #Make predictions on the test dataset
predictions = model.predict(test_features)
predicted_labels = np.argmax(predictions, axis=1)

875/875 [==============================] - 5s 6ms/step


In [14]:
# Create a submission DataFrame
submission = pd.DataFrame({'ImageId': range(1, len(predicted_labels) + 1), 'Label': predicted_labels})

In [15]:
# Save the submission to a CSV file
submission.to_csv('submission.csv', index=False)

In [16]:
# Save the trained model as an .h5 file
model.save('trained_model.keras')